In [1]:
from keras.models import Model
from keras.layers import Input, Bidirectional, Concatenate, Dot, LSTM
from keras.layers.core import Activation, Dense, Lambda
from keras.layers.convolutional import Conv1D, Cropping1D, ZeroPadding1D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import add, multiply
from keras import regularizers, initializers
import keras.backend as kb
from keras.losses import binary_crossentropy, mean_squared_error, categorical_crossentropy
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import numpy as np

Using TensorFlow backend.


In [6]:
input_org = Input(shape=(50, 4), name='input_org')
reverse_layer = Lambda(lambda x: kb.reverse(x, axes=-2))
input_rev = reverse_layer(input_org)

conv_or = input_org
conv_rv = input_rev

num_filter = 64
kernel_size = 7

conv_or = BatchNormalization()(conv_or)
conv_or = Activation('relu')(conv_or)
conv_or = Conv1D(filters=num_filter, kernel_size=kernel_size, padding='same', activation=None)(conv_or)

conv_rv = BatchNormalization()(conv_rv)
conv_rv = Activation('relu')(conv_rv)
conv_rv = Conv1D(filters=num_filter, kernel_size=kernel_size, padding='same', activation=None)(conv_rv)

conv_prod = Dot(axes=-1)([conv_or, conv_rv]) 

stack_layer = Lambda(lambda x: kb.stack(x, axis=-1))
test_stack = stack_layer([conv_prod, conv_prod])

model = Model(input=input_org, output=test_stack)

/Users/alicegao/anaconda2/envs/dg_work/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("la..., inputs=Tensor("in...)`


In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_org (InputLayer)          (None, 50, 4)        0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 50, 4)        0           input_org[0][0]                  
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 50, 4)        16          input_org[0][0]                  
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 50, 4)        16          lambda_5[0][0]                   
__________________________________________________________________________________________________
activation

In [ ]:


input_org = Input(shape=(50, 4), name='input_org')

# input_rev = Input(shape=(51, 4), name='input_rev')  # can also use rev comp
reverse_layer = Lambda(lambda x: kb.reverse(x, axes=-2))
input_rev = reverse_layer(input_org)

conv_prods = []
num_filters = [64, 64, 64, 64, 64]
kernel_sizes = [7, 3, 3, 5, 9]

conv_or = input_org
conv_rv = input_rev

for num_filter, kernel_size in zip(num_filters, kernel_sizes):
    conv_or = BatchNormalization()(conv_or)
    conv_or = Activation('relu')(conv_or)
    conv_or = Conv1D(filters=num_filter, kernel_size=kernel_size, padding='same', activation=None)(conv_or)

    conv_rv = BatchNormalization()(conv_rv)
    conv_rv = Activation('relu')(conv_rv)
    conv_rv = Conv1D(filters=num_filter, kernel_size=kernel_size, padding='same', activation=None)(conv_rv)
    # conv_rv_mid = Cropping1D(25)(conv_rv)

    # # transformation matrix - general
    # conv_or_transform = Conv1D(filters=64, kernel_size=1, activation=None)(conv_or)
    # conv_prod = Dot(axes=-1)([conv_or_transform, conv_rv_mid])
    # conv_prods.append(conv_prod)

    # # replace dot product
    # # by tiling the rev
    # # stack on org
    # # run a mini fully connected net
    # conv_rd_mid_tiled = Lambda(kb.tile, arguments={'n': (1, 51, 1)})(conv_rv_mid)
    # conv_fw_rd = Concatenate(axis=-1)([conv_or, conv_rd_mid_tiled])
    # # TODO hard coded 2 filters
    # # size=1, fully connected along all features at each position
    # hid_fw_rd = Conv1D(filters=2, kernel_size=1, activation='tanh')(conv_fw_rd)
    # conv_prods.append(hid_fw_rd)

    # dot product
    conv_prod = Dot(axes=-1)([conv_or, conv_rv])  # 2D map
    conv_prods.append(conv_prod)
conv_prod_concat = Concatenate(axis=-1)(conv_prods)

# LSTM TODO hard-coded 100 hidden units
hid = Bidirectional(LSTM(100, activation='tanh', recurrent_activation='hard_sigmoid',
                         return_sequences=True))(conv_prod_concat)
output = Conv1D(1, 1, padding='same', activation='sigmoid')(hid)

# # fully connected along feature dimension
# output = Conv1D(1, 1, padding='same', activation='sigmoid')(conv_prod_concat)

# model = Model(input=[input_org, input_rev], output=output)
model = Model(input=input_org, output=output)

